In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sci
pd.set_option('display.max_columns', 100)

# A. Mendapatkan Informasi Dasar Data
  1. load data
  2. load data menjadi dataframe
  3. Cek dataframe
    a. Shape
    b. Head
    c. Tail
    d. Info
    e. Duplikasi (1)
    f. Duplikasi (2)

In [2]:
# 1. load data
train = 'data/train.csv'
test = 'data/test.csv'
desc = 'data/data_description.txt'

In [3]:
# 2. load data as dataframe
df = pd.read_csv(train)

In [6]:
# 3. checking df
# a. shape
df.shape
print(f"Jumlah baris {df.shape[0]}")
print(f"Jumlah kolom {df.shape[1]}")

Jumlah baris 1460
Jumlah kolom 81


In [ ]:
# b. head
df.head(10)

In [ ]:
# c. tail
df.tail(10)

In [ ]:
# d. info
df.info()

In [11]:
# e. cek duplikasi (1)
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1455    False
1456    False
1457    False
1458    False
1459    False
Length: 1460, dtype: bool

In [12]:
# e. cek duplikasi (2)
df[df.duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
